In [1]:
# Importamos Pandas para el manejo de DF
import pandas as pd
# Importamos NumPy para el manejo de vectores
import numpy as np
# Importamos NLTK (Natural Language Tool-Kit)
import nltk
# Importamos el tokenizador de sentencias
from nltk.tokenize import PunktSentenceTokenizer
# Importamos el tokenizador de palabras
from nltk.tokenize import word_tokenize
# Importamos WordNet
from nltk.corpus import wordnet as wn
# Importamos el lexicón de opiniones
from nltk.corpus import opinion_lexicon
# Importamos SentiWordNet
from nltk.corpus import sentiwordnet as swn
# Importamos el algoritmo de WSD simple lesk
from pywsd import simple_lesk
# Importamos las métricas de rendimiento
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Importamos las palabras vacías
from nltk.corpus import stopwords
# Importamos el clasificador Naive Bayes
from nltk.classify import NaiveBayesClassifier
# Importamos el clasificador de máxima entropía
from nltk.classify import MaxentClassifier
# Importamos el corpus de opiniones de películas
from nltk.corpus import movie_reviews

english_sw = set(stopwords.words("english"))

Warming up PyWSD (takes ~10 secs)... took 2.657538890838623 secs.


- **Ejercicio 1:** Se quiere desarrollar un evaluador de sentimientos basado en el uso de un recursos lingüístico externo, en concreto, las listas de palabras de opinión que tiene NLTK.

In [2]:
opinion1 = "Visceral, stunning and relentless film making. Dicaprio's Herculean, almost purely physical performance" \
            "and Hardy's wide eyed intensity coupled with the almost overwhelming beauty of the landscape - those " \
            "trees, the natural light, the sun peeking through the clouds, rendered the proceedings down to savage" \
            "poetry. A hypnotic, beautiful, exhausting film."

opinion2 = "I saw this film on Friday. For the first 40 minutes involving spoken dialogue they need not have " \
            "bothered. For me the dialogue was totally unintelligible with grunting, southern states drawl, " \
            "and coarse accent that made it impossible to understand what they were saying."

opinion3 = "It was a idiotic film that produces a magnificent fascination."

In [3]:
def tokenize_text(text):
    sent_tokenized = PunktSentenceTokenizer().tokenize(text)
    word_tokenized = [word_tokenize(s) for s in sent_tokenized]
    return word_tokenized

In [4]:
def classify(text):
    word_tokenized = tokenize_text(text)
    score = 0
    for s in word_tokenized:
        for w in s:
            if w in opinion_lexicon.positive() and w in opinion_lexicon.negative():
                score += 0
            elif w in opinion_lexicon.positive():
                score += 1
            elif w in opinion_lexicon.negative():
                score -= 1
    if score == 0:
        return "neutro", score
    elif score > 0:
        return "pos", score
    else:
        return "neg", score

In [5]:
# Opinión 1
print(classify(opinion1))

('pos', 1)


In [6]:
# Opinión 2
print(classify(opinion2))

('neg', -4)


In [7]:
# Opinión 3
print(classify(opinion3))

('pos', 1)


---
- **Ejercicio 2:** Se pide lo mismo que en el ejercicio anterior, pero en este caso utilizando SentiWordNet (disponible en NLTK). Esta base de datos proporciona valores positivos y negativos para ciertas palabras en un rango entre -1 y 1. Se puede seguir la misma idea de algoritmo que en el caso anterior, pero hay que tener en cuenta que SentiWordNet nos proporciona puntuaciones para los diferentes sentidos que tiene una palabra. Se puede entonces considerar la puntuación de todos los sentidos de la misma palabra, restando a lo positivo la puntuación negativa. Puede ser interesante que la puntuación global se promedie de acuerdo con el número de sentidos.  Utilizar como entrada las mismas opiniones del ejercicio anterior. ¿El resultado es mejor o peor que el conseguido con el algoritmo del ejercicio 1?

In [8]:
def classify_swn(text):
    word_tokenized = tokenize_text(text)
    total_score = 0
    for s in word_tokenized:
        for w in s:
            senti_synsets = list(swn.senti_synsets(w))
            word_score = 0
            for senti_synset in senti_synsets:
                word_score += senti_synset.obj_score()
            if len(senti_synsets) > 0:
                total_score += word_score / len(senti_synsets)
    return total_score

In [9]:
# Opinión 1
print(classify_swn(opinion1))

27.784353633954698


In [10]:
# Opinión 2
print(classify_swn(opinion2))

27.681840555278054


In [11]:
# Opinión 3
print(classify_swn(opinion3))

7.056547619047619


- **Ejercicio 2.1:** Hacer una variante del ejercicio donde se tengan en cuenta primero la categoría gramatical del token para considerar únicamente los scores de los sentidos que coincidan con la categoría gramatical dada. ¿Ha mejorado el resultado o ha empeorado con respecto a versiones anteriores?

In [12]:
# Función auxiliar para convertir un
# tag generado por nltk.pos_tag()
# en una categ. gramatical de 
# WordNet
def tag2wn(tag):
    if tag.startswith("N"):
        return wn.NOUN
    elif tag.startswith("V"):
        return wn.VERB
    elif tag.startswith("J"):
        return wn.ADJ
    elif tag.startswith("R"):
        return wn.ADV
    else:
        return None

def classify_swn_gram(text):
    word_tokenized = tokenize_text(text)
    tags = [nltk.pos_tag(s) for s in word_tokenized]
    total_score = 0
    for s in tags:
        for w, t in s:
            synsets = list(swn.senti_synsets(w, tag2wn(t)))
            word_score = 0
            for synset in synsets:
                word_score += synset.obj_score()
            if len(synsets) > 0:
                total_score += word_score / len(synsets)
    return total_score

In [13]:
# Opinión 1
print(classify_swn_gram(opinion1))

26.805654761904766


In [14]:
# Opinión 2
print(classify_swn_gram(opinion2))

26.569603892348248


In [15]:
# Opinión 3
print(classify_swn_gram(opinion3))

7.053113553113552


Notamos que la mejora es más bien leve.

---
- **Ejercicio 3:** Se pide extender lo que se ha hecho en los ejercicios anteriores, pero ampliado para todas las opiniones contenidas en un fichero .csv (textsSentimentsPNN.csv), donde cada opinión está anotada con su polaridad (positiva, negativa o neutra). Un ejemplo del formato del archivo se muestra a continuación, en la primera columna estaría el texto de la opinión y en la segunda el sentimiento (Positive, Negative o Neutral).  
 
    Utilizando de nuevo SentiWordNet se pide predecir la polaridad de cada mensaje. Una vez que se tenga la polaridad de cada mensaje, utilizar las métricas del paquete sklearn.metrics de la librería scikit-learn para obtener los valores de accuracy, precisión, recall y f-measure. En definitiva, se quiere poder cuantificar si se están clasificando bien las opiniones según su polaridad. Cuanto más cercano a 1 sea el valor que se obtiene con estas métricas, mejor estará clasificando cada mensaje.

---
- **Ejercicio 4:** Utilizar el corpus de opiniones de películas disponible en NLTK (corpus_reviews). Lo que se quiere es entrenar un clasificador (en este caso NaiveBayesClassifier) para que aprenda sobre opiniones de películas ya anotadas como positivas o negativas, y así, una vez entrenado, poder conocer la polaridad de nuevas opiniones que queramos probar.  
    Se proporciona un código de ejemplo donde se ha partido el conjunto de datos en entrenamiento y test, se ha entrenado el clasificador y se ha evaluado con la parte de test (archivo ClasificadorReviews.py).  
    Lo que se pide realmente en el ejercicio es, una vez se tiene el clasificador entrenado, probarlo para que clasifique cada una de las tres opiniones que se han utilizado en los ejercicios 1 y 2 de este listado. Hay que comprobar si el resultado mejora o empeora.
    Se pueden hacer pruebas de todos los algoritmos propuestos con otras opiniones para comparar.  
    Si se utiliza otro clasificador diferente, ¿podría variar el resultado? Probar a utilizar el clasificador de Máxima Entropía (maxent en NLTK).

In [16]:
negids = movie_reviews.fileids("neg")
posids = movie_reviews.fileids("pos")
print(negids[:5])
print(posids[:5])

['neg/cv000_29416.txt', 'neg/cv001_19502.txt', 'neg/cv002_17424.txt', 'neg/cv003_12683.txt', 'neg/cv004_12641.txt']
['pos/cv000_29590.txt', 'pos/cv001_18431.txt', 'pos/cv002_15918.txt', 'pos/cv003_11664.txt', 'pos/cv004_11636.txt']


In [17]:
negfeats = [(dict((w, True) for w in movie_reviews.words(file)), "neg") for file in negids]
posfeats = [(dict((w, True) for w in movie_reviews.words(file)), "pos") for file in posids]
print(negfeats[:5])
print(posfeats[:5])

[({'plot': True, ':': True, 'two': True, 'teen': True, 'couples': True, 'go': True, 'to': True, 'a': True, 'church': True, 'party': True, ',': True, 'drink': True, 'and': True, 'then': True, 'drive': True, '.': True, 'they': True, 'get': True, 'into': True, 'an': True, 'accident': True, 'one': True, 'of': True, 'the': True, 'guys': True, 'dies': True, 'but': True, 'his': True, 'girlfriend': True, 'continues': True, 'see': True, 'him': True, 'in': True, 'her': True, 'life': True, 'has': True, 'nightmares': True, 'what': True, "'": True, 's': True, 'deal': True, '?': True, 'watch': True, 'movie': True, '"': True, 'sorta': True, 'find': True, 'out': True, 'critique': True, 'mind': True, '-': True, 'fuck': True, 'for': True, 'generation': True, 'that': True, 'touches': True, 'on': True, 'very': True, 'cool': True, 'idea': True, 'presents': True, 'it': True, 'bad': True, 'package': True, 'which': True, 'is': True, 'makes': True, 'this': True, 'review': True, 'even': True, 'harder': True, 'w

In [18]:
negcutoff = int(len(negfeats) * 3/4)
poscutoff = int(len(posfeats) * 3/4)
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]
print(f"Tamaño del conjunto de entrenamiento: {len(trainfeats)}")
print(f"Tamaño del conjunto de test: {len(testfeats)}")

Tamaño del conjunto de entrenamiento: 1500
Tamaño del conjunto de test: 500


In [19]:
# Naive Bayes Classifier
classifier = NaiveBayesClassifier.train(trainfeats)

In [20]:
# Convertimos las opiniones
def preprocess(text):
    wt = word_tokenize(text)
    return dict((w, True) for w in wt)
    

print(f"Opinión 1: {classifier.classify(preprocess(opinion1))}")
print(f"Opinión 2: {classifier.classify(preprocess(opinion2))}")
print(f"Opinión 3: {classifier.classify(preprocess(opinion3))}")

Opinión 1: pos
Opinión 2: neg
Opinión 3: pos


In [21]:
# Accuracy en conj. de test
print(f"Accuracy: {nltk.classify.accuracy(classifier, testfeats)}")

Accuracy: 0.728


In [22]:
# Probamos MaxentClassifier
classifier = MaxentClassifier.train(trainfeats, algorithm="GIS")

  ==> Training (100 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.500
             2          -0.69252        0.953
             3          -0.69190        0.953
             4          -0.69128        0.954
             5          -0.69066        0.954
             6          -0.69005        0.954
             7          -0.68943        0.955
             8          -0.68881        0.955
             9          -0.68820        0.955
            10          -0.68758        0.956
            11          -0.68697        0.957
            12          -0.68636        0.959
            13          -0.68575        0.959
            14          -0.68514        0.960
            15          -0.68453        0.961
            16          -0.68392        0.961
            17          -0.68332        0.961
            18          -0.68271        0.961
            19          -0.68211        0.961
 

In [23]:
print(f"Opinión 1: {classifier.classify(preprocess(opinion1))}")
print(f"Opinión 2: {classifier.classify(preprocess(opinion2))}")
print(f"Opinión 3: {classifier.classify(preprocess(opinion3))}")

Opinión 1: pos
Opinión 2: neg
Opinión 3: pos


In [24]:
# Accuracy en conj. de test
print(f"Accuracy: {nltk.classify.accuracy(classifier, testfeats)}")

Accuracy: 0.79
